In [6]:

'''
core MIP entries, generally singular, and do not change often. 
'''
import json 

# repo_ctx = json.load(open('../context.json'))
# repo_ctx['@depends'] = ['mip-cmor-tables:auxillary/mip_era',]

activity=[] 

from jsonschema import validate,exceptions

'''
cmip6plus/source
'''

def validate_json(data, schema, name = 'none'):
    try:
        validate(instance=data, schema=schema)
        print(f"Validation succeeded: {name}")
    except exceptions.ValidationError as err:
        print("Validation error:", err.message)

In [7]:
schema = {
  "$schema": "http://json-schema.org/draft-07/schema#",
  "title": "CMIP6 Source ID",
  "type": "object",
  "required": [
    "@id",
    "@type",
    "name",
    "label",
    "label_extended",
    "release_year",
    "activity_participation",
    "cohort",
    "institution_id",
    "contributers",
    "license",
    "model-component"
  ],
  "properties": {
    "@id": {
      "type": "string",
      "format": "uri",
      "pattern": "^cmip6plus:source/id/.+$"
    },
    "@type": {
      "type": "string",
      "pattern": "^cmip:source_id$"
    },
    "name": {
      "type": "string"
    },
    "label": {
      "type": "string"
    },
    "label_extended": {
      "type": "string"
    },
    "release_year": {
      "type": "integer",
      "minimum": 2000,
      "maximum": 2100
    },
    "activity_participation": {
      "type": "object",
      "properties": {
        "@set": {
          "type": "array",
          "items": {
            "type": "object",
            "properties": {
              "@id": {
                "type": "string",
                "format": "uri",
                "pattern": "^cmip6plus:activity/.+$"
              }
            },
            "required": ["@id"]
          }
        }
      },
      "required": ["@set"],
      "additionalProperties": False
    },
    "cohort": {
      "type": "string",
      "enum": ["published"]
    },
    "institution_id": {
      "type": "object",
      "properties": {
        "@id": {
          "type": "string",
          "format": "uri",
          "pattern": "^mip-cmor-tables:institutions/.+$"
        }
      },
      "required": ["@id"],
      "additionalProperties": False
    },
    "contributers": {
      "type": "object",
      "properties": {
        "@set": {
          "type": "object"
        }
      },
      "required": ["@set"],
      # "additionalProperties": False
    },
    "license": {
      "type": "object",
      "properties": {
        "@nest": {
          "type": "object",
          "properties": {
            "license:type": {
              "type": "object",
              "properties": {
                "@id": {
                  "type": "string",
                  "format": "uri",
                  "pattern": "^mip-cmor-tables:license/.+$"
                }
              },
              "required": ["@id"],
              "additionalProperties": False
            },
            "license:exceptions_contact": {
              "type": "string"
            },
            "license:source_specific_info": {
              "type": "string"
            }
          },
          "required": ["license:type", "license:exceptions_contact", "license:source_specific_info"],
          "additionalProperties": False
        }
      },
      "required": ["@nest"],
      "additionalProperties": False
    },
    "model-component": {
      "type": "object",
      "properties": {
        "@list": {
          "type": "array",
          "items": {
            "type": "object",
            "properties": {
              "@id": {
                "type": "string",
                "format": "uri",
                "pattern": "^cmip6plus:model/component/.+$"
              }
            },
            "required": ["@id"]
          }
        }
      },
      "required": ["@list"],
      "additionalProperties": False
    }
  },
  "additionalProperties": False
}

loc = f'./id/schema.jsonld'
json.dump(schema, open(loc,'w'),indent=4)

In [8]:
source = json.load(open('../../CMIP6Plus_source_id.json'))['source_id']


In [10]:
lsn = []

for src in source:
    values = source[src]
    license = values['license_info']
    lsn.append(license)
    
    
    
    mc = []
    
    for v , mcomp in values['model-component'].items():
        
        comp = list(mcomp.values())[0].split(" (")[0].lower().replace(' ','_').replace('_','-')
        
        
        if bool(comp) and comp!= 'none':
            
        
            mc.append({"@id":f'cmip6plus:model/component/{comp}'})
            # print(comp, mcomp)
        del comp
    
    data = {
        "@id":"cmip6plus:source/id/"+src.lower(),
        "@type":"source-id",
        "name": src,
        "label":values['label'],
         "label-extended":values['label_extended'],
        "release-year":int(values['release_year']),
        "activity-participation":{"@set":[{"@id":f'cmip6plus:activity/{i.lower()}'} for i in values["activity_participation"]]},
        "cohort": "published",
        "institution-id": {"@id":'mip-cmor-tables:institutions/'+values['institution_id'][0].lower()},
        "contributers":{"@set":{}},
        "license":
            {"@nest":{
                "license:type":{"@id":f"mip-cmor-tables:license/{license['id'].lower().replace(' ','-')}"},
                "license:exceptions_contact":license['exceptions_contact'],
                "license:source_specific_info":license['source_specific_info']
            }},
        "model-component":
            {"@set":mc}
            
        }
    
    

    validate_json(data,schema,src)

    loc = f'./id/{src.lower()}.json'
    json.dump(data, open(loc,'w'),indent=4)
        
        

Validation error: 'label_extended' is a required property
Validation error: 'label_extended' is a required property
Validation error: 'label_extended' is a required property
Validation error: 'label_extended' is a required property
Validation error: 'label_extended' is a required property
Validation error: 'label_extended' is a required property


In [6]:
frame = {
  "@context": {
    "@vocab": "relevant_id",
    "@explicit": False,
  },
  "@type": "source-id:type",
  "name": "",
  "label": "",
  "label_extended": "",
  "release_year": "",
  "activity_participation": {
    "@set": [{
        "@context": {
        "@vocab": " activity-id:",
        "@explicit": True
        },
  }]
  },
  "cohort": "",
  "institution_id": {
    "@context": {
      "@vocab": "institution",
      "@explicit": True
    },
  },
  "contributers": {"@set": {}},
  "license": {
    "@nest": {
      "type": {
        "@context": {
          "@vocab": "license:",
          "@explicit": True
        },
        "@id": "license:type"
      },
      "exceptions_contact": {"@id": "@lists.nasa.gov <- cmip-giss-l"},
      "source_specific_info": ""
    }
  },
  "model-component": {
    "@list": [
      {
        "@context": {
        "@vocab": "model-component:",
        "@explicit": True
        },
  }
    ]
  }
}


In [7]:
ctx = repo_ctx['@context'].copy()
ctx.update({"@vocab":"source-id:",
            "@depends":['mip-cmor-tables:license','model/components','activity','source/type'],
            "license":{"@vocab":'license'}})

loc = f'./id/context.json'
json.dump({'@context':ctx}, open(loc,'w'),indent=4)

In [8]:
''' Source type '''

st = {
        "AER":"aerosol treatment in an atmospheric model where concentrations are calculated based on emissions, transformation, and removal processes (rather than being prescribed or omitted entirely)",
        "AGCM":"atmospheric general circulation model run with prescribed ocean surface conditions and usually a model of the land surface",
        "AOGCM":"coupled atmosphere-ocean global climate model, additionally including explicit representation of at least the land and sea ice",
        "BGC":"biogeochemistry model component that at the very least accounts for carbon reservoirs and fluxes in the atmosphere, terrestrial biosphere, and ocean",
        "CHEM":"chemistry treatment in an atmospheric model that calculates atmospheric oxidant concentrations (including at least ozone), rather than prescribing them",
        "ISM":"ice-sheet model that includes ice-flow",
        "LAND":"land model run uncoupled from the atmosphere",
        "OGCM":"ocean general circulation model run uncoupled from an AGCM but, usually including a sea-ice model",
        "RAD":"radiation component of an atmospheric model run 'offline'",
        "SLAB":"slab-ocean used with an AGCM in representing the atmosphere-ocean coupled system"
    }


source_types = []

for i,j in st.items():
    data = {
        "@id":"cmip6plus:source/type/"+i.lower(),
        "@type":"mip:source-type",
        "name":i,
        "description":j
    }
        
    # source_types.append(data)
    

    loc = f'./type/{i.lower()}.json'
    json.dump(data, open(loc,'w'),indent=4)



FileNotFoundError: [Errno 2] No such file or directory: './type/aer.json'

In [ ]:
# ctx = repo_ctx['@context'].copy()
# ctx.update({"@vocab":"source-type:"})

# loc = f'./type/context.json'
# json.dump({'@context':ctx}, open(loc,'w'),indent=4)

In [ ]:
frame = {
{
  "@context": {
    "@vocab": "source-type:"
  },
  "@type": "mip:source-type",
  "name": "",
  "description": ""
}

}

loc = f'./type/frame.json'
json.dump(frame, open(loc,'w'),indent=4)
import os 
os.system('rm context.json')

TypeError: unhashable type: 'dict'